# M2177.003100 Deep Learning <br>Assignment #5 Part 1: Implementing and Training a Deep Q-Network

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Hyemi Jang, November 2018

In this notebook, you will implement one of famous reinforcement learning algorithm, Deep Q-Network (DQN) of DeepMind. <br>
The goal here is to understand a basic form of DQN [1, 2] and learn how to use OpenAI Gym toolkit [3].<br>
You need to follow the instructions to implement the given classes.

1. [Play](#play) ( 50 points )

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **two parts of the assignment**, run the *CollectSubmission.sh* script with your **Team number** as input argument. <br>
This will produce a zipped file called *[Your team number].tar.gz*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* &nbsp; Team_#)

### Some helpful references for assignment #4 :
- [1] Mnih, Volodymyr, et al. "Playing atari with deep reinforcement learning." arXiv preprint arXiv:1312.5602 (2013). [[pdf]](https://www.google.co.kr/url?sa=t&rct=j&q=&esrc=s&source=web&cd=3&cad=rja&uact=8&ved=0ahUKEwiI3aqPjavVAhXBkJQKHZsIDpgQFgg7MAI&url=https%3A%2F%2Fwww.cs.toronto.edu%2F~vmnih%2Fdocs%2Fdqn.pdf&usg=AFQjCNEd1AJoM72DeDpI_GBoPuv7NnVoFA)
- [2] Mnih, Volodymyr, et al. "Human-level control through deep reinforcement learning." Nature 518.7540 (2015): 529-533. [[pdf]](https://www.nature.com/nature/journal/v518/n7540/pdf/nature14236.pdf)
- [3] OpenAI GYM website [[link]](https://gym.openai.com/envs) and [[git]](https://github.com/openai/gym)

## 0. OpenAI Gym

OpenAI Gym is a toolkit to support diverse environments for developing reinforcement learning algorithms. You can use the toolkit with Python as well as TensorFlow. Installation guide of OpenAI Gym is offered by [this link](https://github.com/openai/gym#installation) or just type the command "pip install gym" (as well as "pip install gym[atari]" for Part2). 

After you set up OpenAI Gym, you can use APIs of the toolkit by inserting <font color=red>import gym</font> into your code. In this assignment, you must build one of famous reinforcement learning algorithms whose agent can run on OpenAI Gym environments. Please check how to use APIs such as funcions interacting with environments in the followings.

In [1]:
#import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 
import gym
import numpy as np
import os

In [2]:
# Make an environment instance of CartPole-v0.
env = gym.make('CartPole-v0')

# Before interacting with the environment and starting a new episode, you must reset the environment's state.
state = env.reset()

# Uncomment to show the screenshot of the environment (rendering game screens)
# env.render() 

# You can check action space and state (observation) space.
num_actions = env.action_space.n
state_shape = env.observation_space.shape
print(num_actions)
print(state_shape)

# "step" function performs agent's actions given current state of the environment and returns several values.
# Input: action (numerical data)
#        - env.action_space.sample(): select a random action among possible actions.
# Output: next_state (numerical data, next state of the environment after performing given action)
#         reward (numerical data, reward of given action given current state)
#         terminal (boolean data, True means the agent is done in the environment)
next_state, reward, terminal, info = env.step(env.action_space.sample())

2
(4,)


## 1. Implement a DQN agent
## 1) Overview of implementation in the notebook

The assignment is based on a method named by Deep Q-Network (DQN) [1,2]. You could find the details of DQN in the papers. The followings show briefly architecture of DQN and its training computation flow.

- (Pink flow) Play an episode and save transition records of the episode into a replay memory.
- (Green flow) Train DQN so that a loss function in the figure is minimized. The loss function is computed using main Q-network and Target Q-network. Target Q-network needs to be periodically updated by copying the main Q-network.
- (Purple flow) Gradient can be autonomously computed by tensorflow engine, if you build a proper optimizer.

![](image/architecture.png)

There are major 4 components, each of which needs to be implemented in this notebook. The Agent class must have an instance(s) of each class (Environment, DQN, ReplayMemory).
- Environment
- DQN 
- ReplayMemory
- Agent

![](image/components.png)



## 2) Design classes

In the code cells, there are only names of functions which are used in TA's implementation and their brief explanations. <font color='green'>...</font> means that the functions need more arguments and <font color='green'>pass</font> means that you need to write more codes. The functions may be helpful when you do not know how to start the assignment. Of course, you could change the functions such as deleting/adding functions or extending/reducing roles of the classes, <font color='red'> just keeping the existence of the classes</font>.

### Environment class

In [3]:
import argparse
import random

In [4]:
class Environment(object):
	def __init__(self, args):
		# Make an environment instance of CartPole-v0.
		self._env = gym.make(args.env_name)
		self._initialized = False

		# Uncomment to show the screenshot of the environment (rendering game screens)
		# env.render() 

	def random_action(self):
		# Return a random action.
		return self._env.action_space.sample()

	def get_action_space_count(self):
		return self._env.action_space.n

	def get_state_shape(self):
		return self._env.observation_space.shape

	def render_worker(self):
		# If display in your option is true, do rendering. Otherwise, do not.
		pass

	def new_episode(self):
		# Sart a new episode and return the first state of the new episode.
		self._initialized = True
		state = self._env.reset()

		return state

	def act(self, action):
		assert(self._initialized)
		assert(self._env.action_space.contains(action))
		# Perform an action which is given by input argument and return the results of acting.
		# "step" function performs agent's actions given current state of the environment and returns several values.
		# Input: action (numerical data)
		#        - env.action_space.sample(): select a random action among possible actions.
		# Output: next_state (numerical data, next state of the environment after performing given action)
		#         reward (numerical data, reward of given action given current state)
		#         terminal (boolean data, True means the agent is done in the environment)
		next_state, reward, terminal, info = self._env.step(action)
		return next_state, reward, terminal

### ReplayMemory class

In [5]:
class ReplayMemory(object):
	def __init__(self, args):
		self._args = args
		self._memory = []
		
	def add(self, s, a, r, t, s_):
		# Add current_state, action, reward, terminal, (next_state which can be added by your choice). 
		self._memory.append([s, a, r, t, s_])
		if len(self._memory) > self._args.replay_memory_size:
			self._memory = self._memory[-self._args.replay_memory_size:]

	def mini_batch(self):
		# Return a mini_batch whose data are selected according to your sampling method. (such as uniform-random sampling in DQN papers)
		assert(len(self._memory) >= self._args.re_valid_count)

		datas = []

		while(len(datas) < self._args.re_batch_size):
			datas.append(self._memory[random.randrange(len(self._memory))])

		return np.array(datas)

	def count(self):
		return len(self._memory)
        

### DQN class

In [6]:
class DQN(object):
	def __init__(self, args, sess, env):
		self.sess = sess
		self._env = env
		self._args = args

		self.prediction_Q = self.build_network('pred')
		self.target_Q = self.build_network('target')
		self.optimizer = self.build_optimizer()

		self._copy_op = []
		pred_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='pred')
		target_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='target')
		
		for pred_var, target_var in zip(pred_vars, target_vars):
			self._copy_op.append(target_var.assign(pred_var.value()))

	def build_network(self, name):
		# Make your a deep neural network
		with tf.variable_scope(name):
			state_dim = self._env.get_state_shape()[0]
			action_dim = self._env.get_action_space_count()
			drop_out_rate = 0.7

			t_input_s = tf.placeholder(tf.float32, shape=[None, state_dim])
#			t_input_a = tf.placeholder(tf.float32, shape=[None, action_dim])
			t_is_train = tf.placeholder_with_default(True, shape=())

#			flow = tf.concat([t_input_s, t_input_a], axis=1)
			flow = t_input_s
			flow = tf.layers.dense(flow, 486, activation=tf.nn.relu)
			flow = tf.layers.dense(flow, 618, activation=tf.nn.relu)
			flow = tf.layers.dense(flow, 394, activation=tf.nn.relu)
#			flow = tf.layers.batch_normalization(flow)
#			flow = tf.layers.dropout(flow, rate=drop_out_rate, training=t_is_train)
#			flow = flow + flow * (1.0 / (1.0-drop_out_rate) - 1.0) * tf.to_float(t_is_train)
			t_reward = tf.layers.dense(flow, action_dim)

		return (t_input_s, t_is_train, t_reward)

	def build_optimizer(self):
		# Make your optimizer 
		action_dim = self._env.get_action_space_count()
		lr = self._args.learning_rate

		t_input_tq = tf.placeholder(tf.float32, shape=[None, 1])
		t_input_actions = tf.placeholder(tf.int32, shape=[None, 1])
		t_input_s, t_is_train, t_reward = self.prediction_Q

		t_pred = tf.reduce_sum(tf.multiply(t_reward, tf.reshape(tf.one_hot(t_input_actions, action_dim, dtype=tf.float32), [-1, action_dim])), 1, keepdims=True)
		
		t_loss = tf.reduce_mean(tf.square(t_input_tq - t_reward), axis=1)
		t_optimizer = tf.train.AdamOptimizer(lr).minimize(t_loss) 

		return (t_input_s, t_input_actions, t_is_train, t_input_tq, t_loss, t_optimizer)

	def train_network(self, samples):
		# Train the prediction_Q network using a mini-batch sampled from the replay memory
		#self._memory.append([s, a, r, t, s_])
		t_s = np.stack(samples[:,0])
		t_a = np.expand_dims(np.array(samples[:,1]), axis=1)
		t_r = np.array(samples[:,2])
		t_t = np.array(samples[:,3])
		t_s_ = np.stack(samples[:,4])

		target_rewards = self.predict(t_s_, self.target_Q)
		max_target_reward = np.max(target_rewards, axis=1)

		gamma = self._args.gamma

		t_y = []
		for i in range(t_t.shape[0]):
			if t_t[i] == True:
				t_y.append(t_r[i])
			else:
				t_y.append(t_r[i] + gamma * max_target_reward[i])

		t_y = np.expand_dims(np.stack(t_y), axis=1)

		t_input_s, t_input_actions, t_is_train, t_input_tq, t_loss, t_optimizer = self.optimizer

		feed_dict = {
			t_input_s: t_s,
			t_input_actions: t_a,
			t_is_train: True,
			t_input_tq: t_y,
		}

		loss, _ = self.sess.run([t_loss, t_optimizer], feed_dict=feed_dict)

		return np.mean(loss)

	def update_target_network(self):
		self.sess.run(self._copy_op)
		
	def predict(self, t_s, Q_net):
		t_input_s, t_is_train, t_reward = Q_net

		feed_dict = {
			t_input_s: t_s,
			t_is_train: False,
		}
		rewards, = self.sess.run([t_reward], feed_dict=feed_dict)

		return rewards

	def predict_action(self, s):
		t_s = np.array([s])
		rewards = self.predict(t_s, self.target_Q)

		return np.argmax(np.squeeze(rewards, axis=0))

### Agent class

In [7]:
class Agent(object):
	def __init__(self, args, sess, memory, dqn, env):
		self.saver = tf.train.Saver()

		self._args = args
		self._sess = sess
		self._memory = memory
		self._dqn = dqn
		self._env = env

		self._train_count = 0

	def get_epsilon(self):
		return max(1.0 - self._train_count / self._args.epsilon_factor, 0.1)

	def select_action(self, state):
		# Select an action according ε-greedy. You need to use a random-number generating function and add a library if necessary.

		epsilon = self.get_epsilon()

		if random.random() < epsilon:
			return self._env.random_action()

		return self._dqn.predict_action(state)

	def train(self):
		# Train your agent which has the neural nets.
		# Several hyper-parameters are determined by your choice (Options class in the below cell)
		# Keep epsilon-greedy action selection in your mind

		s = self._env.new_episode()

		losses = []
		steps = []
		for step in range(self._args.max_step_per_episode):
			a = self.select_action(s)
			s_, r, t = self._env.act(a)

			# Backup
			self._memory.add(s, a, r, t, s_)

			# Train DQN
			if self._memory.count() >= self._args.re_valid_count:
				loss = self._dqn.train_network(self._memory.mini_batch())
				losses.append(loss)
				self._train_count += 1

			# Update target Q-Network
			if (step+1) % self._args.target_q_update_step == 0:
				self._dqn.update_target_network()

			# The end of this episode
			if t:
				steps.append(step)
				break

			# next state
			s = s_

		return np.mean(losses), np.mean(steps)

	def play(self):
		# Test your agent 
		# When performing test, you can show the environment's screen by rendering,
		total_reward = 0.0
		s = self._env.new_episode()
		t = False
		for _ in range(self._args.max_step_per_episode):
			if t:	break
			a = self._dqn.predict_action(s)
			s, r, t = self._env.act(a)
			total_reward += r

		return total_reward

	def save(self):
		checkpoint_dir = 'cartpole'
		if not os.path.exists(checkpoint_dir):
			os.mkdir(checkpoint_dir)
		self.saver.save(self._sess, os.path.join(checkpoint_dir, 'trained_agent'))

	def load(self):
		checkpoint_dir = 'cartpole'
		self.saver.restore(self._sess, os.path.join(checkpoint_dir, 'trained_agent'))

## 2. Train your agent 

Now, you train an agent to play CartPole-v0. Options class is the collection of hyper-parameters that you can choice. Usage of Options class is not mandatory.<br>
The maximum value of total reward which can be aquired from one episode is 200. 
<font color='red'>**You should show learning status such as the number of observed states and mean/max/min of rewards frequently (for instance, every 100 states).**</font>

In [8]:
def train():

	best_reward = -1

	with tf.Session(config=config) as sess:
		env = Environment(args)
		memory = ReplayMemory(args)
		dqn = DQN(args, sess, env)
		myAgent = Agent(args, sess, memory, dqn, env)
		tf.global_variables_initializer().run()
		myAgent.load()
		steps = []
		for epi in range(args.train_episode_count):
			loss, step = myAgent.train()
			steps.append(step)

			if epi % 10 == 0:
				play_rewards = []
				for _ in range(20):
					play_reward = myAgent.play()
					play_rewards.append(play_reward)
				current_reward = np.mean(play_rewards)

				if best_reward <= current_reward:
					best_reward = current_reward
					print('model saved! - best reward({:.4f})'.format(best_reward))
					myAgent.save()

				print('epi {} - loss({:.6f}) train_steps({:.2f}) epsilon({:.4f}) and play reward is {:.2f}'.format(epi, loss, np.mean(steps), myAgent.get_epsilon(), current_reward))
				steps = []

	print('best reward is {}!!!'.format(best_reward))

In [9]:
parser = argparse.ArgumentParser(description="CartPole")
parser.add_argument('--env-name', default='CartPole-v0', type=str, help="Environment")
parser.add_argument('--re-batch-size', default=32, type=int, help="ReplayMemory minibatch size")
parser.add_argument('--re-valid-count', default=8, type=int, help="Minimum memory size to generate minibatch")
parser.add_argument('--replay-memory-size', default=1000000, type=int, help="Capacity of the replaymemeory")
parser.add_argument('--target-q-update-step', default=5, type=int, help="Update period for target Q-net")
parser.add_argument('--gamma', default=0.99, type=float, help="Decay factor of rewards for the future")
parser.add_argument('--learning-rate', default=1e-3, type=float, help="Learning rate of optimizer")

parser.add_argument('--train-episode-count', default=60000, type=int, help="How many epis to be trained")
parser.add_argument('--max-step-per-episode', default=1000, type=int, help="Maximum steps for an episode")
parser.add_argument('--epsilon-factor', default=1200000.0, type=float, help="Epsilon factor")
parser.add_argument('-f', default='none', type=str)

args = parser.parse_args()

# tf config
config = tf.ConfigProto()
# If you use a GPU, uncomment
config.log_device_placement = False
config.gpu_options.allow_growth = True

## <a name="play"></a> 3. Test the trained agent ( 15 points )

Now, we test your agent and calculate an average reward of 20 episodes.
- 0 <= average reward < 50 : you can get 0 points
- 50 <= average reward < 100 : you can get 10 points
- 100 <= average reward < 190 : you can get 35 points
- 190 <= average reward <= 200 : you can get 50 points

In [10]:
tf.reset_default_graph() 
with tf.Session(config=config) as sess:
    args = parser.parse_args() # You set the option of test phase
    env = Environment(args)
    memory = ReplayMemory(args)
    dqn = DQN(args, sess, env)
    myAgent = Agent(args, sess, memory, dqn, env)
    tf.global_variables_initializer().run()
    
    myAgent.load()
    rewards = []
    for i in range(20):
        r = myAgent.play() # play() returns the reward cumulated in one episode
        rewards.append(r)
    mean = np.mean(rewards)
    print(rewards)
    print(mean)

INFO:tensorflow:Restoring parameters from cartpole/trained_agent
[200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0]
200.0


## Dear TA

혹시 test를 다시 해보시고 싶으시면 모든 셀을 다시 재시작하여 re-run해주세요
Enviroment나 기타 위의 셀에서 정의된 것들이 필요하기 때문에 부득이합니다.
re-run시에도 training이 다시 일어나지 않도록 training 부르는 코드는 일부러 빼두었습니다.